# MGP and MI exchange volumes
The dataset *GME_MGP_MI_QUANTITA* refers to the total volume traded (both sell and buy) for each hours in the MGP and MI markets.

In [1]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])

volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,FLOWDATE,FLOWHOUR,MARKET,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
24,20241004,1,MGP,26381.579,26381.579,2124.022,4534.366,14885.499,763.908,1547.272,...,None,0.0,2909.0,280.0,0.0,234.0,1700.0,23947.579,26381.579,2024-10-04 18:22:30.000
48,20241004,1,MI1,1218.268,1125.417,89.741,120.655,605.125,14.501,46.858,...,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2024-10-04 18:22:35.400
72,20241004,1,MI2,502.341,629.282,18.932,129.258,240.581,21.865,55.090,...,None,0.0,3.7,0.0,0.0,0.0,0.0,NaN,NaN,2024-10-04 18:22:39.830
25,20241004,2,MGP,25258.886,25256.182,2035.433,4339.167,14229.233,731.432,1453.404,...,None,0.0,2898.5,280.0,0.0,234.0,1700.0,22824.886,25258.886,2024-10-04 18:22:30.003
49,20241004,2,MI1,1443.012,1400.190,74.013,116.721,958.660,11.576,42.725,...,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2024-10-04 18:22:35.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,20241024,20,MGP,39618.333,40718.566,3364.848,7064.317,22075.210,1019.076,2223.523,...,None,0.0,3239.0,0.0,0.0,0.0,2772.0,37946.566,40205.333,2024-10-23 13:10:40.527
1548,20241024,21,MGP,36959.831,38017.147,3035.062,6541.118,20632.636,968.576,2152.324,...,None,0.0,3288.0,0.0,0.0,390.0,2832.0,34823.831,37545.831,2024-10-23 13:10:40.527
1549,20241024,22,MGP,33794.538,33849.279,2782.818,6103.470,19104.842,908.811,1966.632,...,None,0.0,2959.0,507.0,0.0,363.0,2550.0,30565.524,33794.538,2024-10-23 13:10:40.527
1550,20241024,23,MGP,30623.797,30727.897,2550.319,5557.782,17231.909,836.810,1815.437,...,None,0.0,2959.0,507.0,0.0,363.0,2550.0,27210.797,30623.797,2024-10-23 13:10:40.527


In [2]:
market_labels = volumes['MARKET'].unique()

print(market_labels)

['MGP' 'MI1' 'MI2' 'MI3']


In [3]:
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

In [4]:
mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
ORAINI,,,,,,,,,,,,,,,,,,,,,
2024-10-04 00:00:00,26381.579,26381.579,2124.022,4534.366,14885.499,763.908,1547.272,1708.231,0.0,None,...,None,0.0,2909.0,280.0,0.0,234.0,1700.0,23947.579,26381.579,2024-10-04 18:22:30.000
2024-10-04 01:00:00,25258.886,25256.182,2035.433,4339.167,14229.233,731.432,1453.404,1641.941,0.0,None,...,None,0.0,2898.5,280.0,0.0,234.0,1700.0,22824.886,25258.886,2024-10-04 18:22:30.003
2024-10-04 02:00:00,24804.218,24779.340,1997.856,4247.864,14015.772,719.020,1414.144,1595.602,0.0,None,...,None,0.0,2877.0,280.0,0.0,234.0,1700.0,22370.218,24804.218,2024-10-04 18:22:30.003
2024-10-04 03:00:00,24457.711,24457.711,2031.358,4139.501,13743.992,716.087,1395.746,1571.391,0.0,None,...,None,0.0,3022.0,679.0,0.0,265.0,1651.0,22269.025,24457.711,2024-10-04 18:22:30.007
2024-10-04 04:00:00,24669.233,24669.233,2009.075,4166.908,13901.838,720.231,1448.271,1591.483,0.0,None,...,None,0.0,3080.0,679.0,0.0,265.0,1700.0,22292.381,24669.233,2024-10-04 18:22:30.007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-24 19:00:00,39618.333,40718.566,3364.848,7064.317,22075.210,1019.076,2223.523,2432.077,0.0,None,...,None,0.0,3239.0,0.0,0.0,0.0,2772.0,37946.566,40205.333,2024-10-23 13:10:40.527
2024-10-24 20:00:00,36959.831,38017.147,3035.062,6541.118,20632.636,968.576,2152.324,2305.711,0.0,None,...,None,0.0,3288.0,0.0,0.0,390.0,2832.0,34823.831,37545.831,2024-10-23 13:10:40.527
2024-10-24 21:00:00,33794.538,33849.279,2782.818,6103.470,19104.842,908.811,1966.632,2109.113,0.0,None,...,None,0.0,2959.0,507.0,0.0,363.0,2550.0,30565.524,33794.538,2024-10-23 13:10:40.527
